In [20]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import os 
import time
from splinter import Browser
from urllib.parse import urlsplit
import requests
import lxml
import html5lib


In [3]:
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
news_url = 'https://mars.nasa.gov/news/'
browser.visit(news_url)

html = browser.html
soup = bs(html,'html.parser')

In [5]:
# save the latest article from nasa about mars
news_title = soup.find('div',class_='content_title').text
news_text = soup.find('div', class_='article_teaser_body').text
print(f'news_title: {news_title}')


news_title: NASA Invites Students to Name Mars 2020 Rover


In [6]:
print(f'news_text: {news_text}')

news_text: Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.


In [7]:
img_url_featured = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(img_url_featured)

# https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars

html_image = browser.html
soup = bs(html_image, 'html.parser')

featured_img_url = soup.find('article')['style'].replace('background-image: url(','').replace(');', '')[1:-1]
main_url = 'https://www.jpl.nasa.gov'
featured_image_url = main_url + featured_img_url
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18816-1920x1200.jpg'

In [8]:
# visit url to collect Mars weather
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)

# scrape the latest weather with bs
weather_html = browser.html
soup = bs(weather_html, 'html.parser')
# mars_weather = soup.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
# mars_weather_all = soup.find_all("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
mars_weather_all = soup.find_all("div", class_="js-tweet-text-container")
# print(mars_weather_all)
for tweet in mars_weather_all:
    mars_tw = tweet.find("p", class_="TweetTextSize").text
    if "InSight sol" in mars_tw:
        print(mars_tw)
        break


InSight sol 265 (2019-08-25) low -99.4ºC (-146.9ºF) high -26.3ºC (-15.3ºF)
winds from the SSE at 5.3 m/s (12.0 mph) gusting to 16.1 m/s (35.9 mph)
pressure at 7.50 hPapic.twitter.com/9YLawm67zS


In [13]:
# set url where we will scrape Mars facts


facts_url = 'http://space-facts.com/mars/'

scrape_table = pd.read_html(facts_url)
# scrape_table[0]
# scrape_table[1]
fact_table = scrape_table[1]
fact_table.columns=['Mars-Fact','Value']
fact_table

,Mars-Fact,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [15]:
# html_fact_table= fact_table.to_html('mars_facts.html') 
html_fact_table= fact_table.to_html() 
html_fact_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars-Fact</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC

In [16]:
browser.quit()

In [23]:

executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
# set url where we will scrape hemispheres images
base_url = 'https://astrogeology.usgs.gov'
HemiUrl = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

# Retrieve page with the requests module
response = requests.get(HemiUrl)

# create bs object and visit url
soup = bs(response.text, 'html.parser')
browser.visit(HemiUrl)

# parse html with bs
html = browser.html
soup = bs(html, 'html.parser')

hemi_results = soup.find_all('div', class_='item')

# Loop through returned results
hemi_image_urls = []

In [27]:
for himage in hemi_results:
    title = himage.find('h3').text
    
    # image link
    partial_link = himage.find('a', class_='itemLink product-item')['href']
    full_link = base_url + partial_link
    
    browser.visit(full_link)

    # parse html with bsbase_url
    html = browser.html
    soup = bs(html, 'html.parser')
    
    hemi_url = base_url+soup.find('img', class_='wide-image')['src']
    
    hemi_image_urls.append({'title' : title,
                            'hemi_img_url' : hemi_url })
    
hemi_image_urls


[{'title': 'Cerberus Hemisphere Enhanced',
  'hemi_img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'hemi_img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'hemi_img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'hemi_img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]

In [28]:
browser.quit()